In [3]:
# USER OPTIONS
# define tokenizer name - should be one among the keys in the cell below
tokenizer_name = 'ChordSymbolTokenizer' # or any other name from the keys in tokenizers dictionary
# tokenizer_name = 'RootTypeTokenizer'
# tokenizer_name = 'PitchClassTokenizer'
# folder to xmls
val_dir = '/media/maindisk/maximos/data/hooktheory_test'
# val_dir = '/media/maindisk/maximos/data/gjt_melodies/Library_melodies'
# val_dir = '/media/datadisk/datasets/gjt_melodies/Library_melodies'
# val_dir = '/media/maximos/9C33-6BBD/data/gjt_melodies/Library_melodies'

# define batch size depending on GPU availability / status
batchsize = 16
# select device name - could be 'cpu', 'cuda', 'coda:0', 'cuda:1'...
device_name = 'cuda'

In [4]:
from data_utils import SeparatedMelHarmMarkovDataset
import os
import numpy as np
from harmony_tokenizers_m21 import ChordSymbolTokenizer, RootTypeTokenizer, \
    PitchClassTokenizer, RootPCTokenizer, GCTRootPCTokenizer, \
    GCTSymbolTokenizer, GCTRootTypeTokenizer, MelodyPitchTokenizer, \
    MergedMelHarmTokenizer
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import BartForConditionalGeneration, BartConfig, DataCollatorForSeq2Seq
from tqdm import tqdm
from models import TransGraphVAE
import csv

In [5]:
tokenizers = {
    'ChordSymbolTokenizer': ChordSymbolTokenizer,
    'RootTypeTokenizer': RootTypeTokenizer,
    'PitchClassTokenizer': PitchClassTokenizer,
    'RootPCTokenizer': RootPCTokenizer,
    'GCTRootPCTokenizer': GCTRootPCTokenizer,
    'GCTSymbolTokenizer': GCTSymbolTokenizer,
    'GCTRootTypeTokenizer': GCTRootTypeTokenizer
}

In [6]:
melody_tokenizer = MelodyPitchTokenizer.from_pretrained('saved_tokenizers/MelodyPitchTokenizer')
harmony_tokenizer = tokenizers[tokenizer_name].from_pretrained('saved_tokenizers/' + tokenizer_name)

tokenizer = MergedMelHarmTokenizer(melody_tokenizer, harmony_tokenizer)

model_path = 'saved_models/bart/' + tokenizer_name + '/' + tokenizer_name + '.pt'

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
bart_config = BartConfig(
    vocab_size=len(tokenizer.vocab),
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    decoder_start_token_id=tokenizer.bos_token_id,
    forced_eos_token_id=tokenizer.eos_token_id,
    max_position_embeddings=512,
    encoder_layers=8,
    encoder_attention_heads=8,
    encoder_ffn_dim=512,
    decoder_layers=8,
    decoder_attention_heads=8,
    decoder_ffn_dim=512,
    d_model=512,
    encoder_layerdrop=0.3,
    decoder_layerdrop=0.3,
    dropout=0.3
)

bart = BartForConditionalGeneration(bart_config)

if torch.cuda.is_available():
    checkpoint = torch.load(model_path, weights_only=True)
else:
    checkpoint = torch.load(model_path, map_location="cpu", weights_only=True)
bart.load_state_dict(checkpoint)

/home/maximos/.local/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


<All keys matched successfully>

In [9]:
test_dir = '/mnt/ssd2/maximos/data/hooktheory_test'
test_dataset = SeparatedMelHarmMarkovDataset(test_dir, tokenizer, max_length=512, num_bars=64)

# Data collator for BART
def create_data_collator(tokenizer, model):
    return DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)
# end create_data_collator

In [10]:
collator = create_data_collator(tokenizer, model=bart)
valloader = DataLoader(test_dataset, batch_size=1, shuffle=True, collate_fn=collator)

In [11]:
config = {
    'hidden_dim_LSTM': 512,
    'hidden_dim_GNN': 256,
    'latent_dim': 512,
    'condition_dim': 256,
    'use_attention': False
}

model = TransGraphVAE(transformer=bart, device=device, tokenizer=tokenizer, **config)
model.to(device)
model.eval()

TransGraphVAE(
  (transformer): BartForConditionalGeneration(
    (model): BartModel(
      (shared): BartScaledWordEmbedding(545, 512, padding_idx=1)
      (encoder): BartEncoder(
        (embed_tokens): BartScaledWordEmbedding(545, 512, padding_idx=1)
        (embed_positions): BartLearnedPositionalEmbedding(514, 512)
        (layers): ModuleList(
          (0-7): 8 x BartEncoderLayer(
            (self_attn): BartSdpaAttention(
              (k_proj): Linear(in_features=512, out_features=512, bias=True)
              (v_proj): Linear(in_features=512, out_features=512, bias=True)
              (q_proj): Linear(in_features=512, out_features=512, bias=True)
              (out_proj): Linear(in_features=512, out_features=512, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=512, out_features=512, bias=True)
            (fc2): Linear(in_feat

In [12]:
with torch.no_grad():
    with tqdm(valloader, unit='batch') as tepoch:
        tepoch.set_description(f'Running')
        print(tepoch)
        for batch in tepoch:
            input_ids = batch['input_ids'].to(device)
            transitions = batch['transitions'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            output_tokens = []
            output_recon_tokens = []
            print(input_ids)
            print(transitions)
            outputs = model(input_ids, transitions, encoder_attention=attention_mask, generate_max_tokens=500)
            for i in outputs['generated_ids'][0]:
                output_tokens.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )
            for i in outputs['generated_recon_ids'][0]:
                output_recon_tokens.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )
            break

Running:   0%|          | 0/1520 [00:00<?, ?batch/s]/home/maximos/.local/lib/python3.11/site-packages/music21/stream/base.py:3694: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)
/home/maximos/.local/lib/python3.11/site-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Running:   0%|          | 0/1520 [00:00<?, ?batch/s]
tensor([[  2,   6, 180,  95,  55, 101,  55, 107,  55, 111,  57, 119,  57,   6,
          95,  58, 101,  57, 107,  53, 111,  55, 119,   4, 123,  53,   6,  95,
          55, 101,  55, 107,  55, 111,  57, 119,  57,   6,  95,  58, 101,  58,
         107,  55, 111,  58, 119,  60]], device='cuda:0')
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0')


Running:   0%|          | 0/1520 [00:02<?, ?batch/s]


In [13]:
print(output_tokens)

['<h>', 'C:sus2', 'C:5', 'F:dim7', 'G:min13', 'D:7(#9)', 'P:22', 'A:maj13', 'D#:7(b13)', 'C:dim7', 'G#:maj9', 'F:dim7', 'G:sus4', 'ts_19x8', 'A:dim7', 'G#:min11', 'P:25', 'P:44', 'G:sus4', 'D:7(b13)', 'G#:7(#11)', '<emp>', 'G:minmaj7', 'D#:maj13', 'position_9x83', 'P:85', 'P:85', 'position_9x83', 'G#:dim7', 'ts_7x8', 'P:85', 'G#:minmaj7', 'D:sus4', 'F:min13', 'ts_15x8', '<emp>', 'P:21', 'G:13', 'D#:1', 'G#:maj9', 'position_8x66', 'P:81', 'D:aug', 'position_5x16', 'D:9', 'ts_19x8', 'D#:dim', 'C#:dim', 'C#:dim', 'C:dim7', 'A:maj13', 'C#:sus4', 'D#:7(b13)', 'position_9x83', 'position_7x75', 'C:min6', 'C:min11', 'F#:sus4', 'position_9x83', 'C#:hdim7', 'A:maj13', 'position_9x83', '<rest>', 'E:', 'position_3x66', 'G#:min11', 'G:13', 'P:85', 'G#:dim7', 'F#:9', 'A:maj7', 'position_2x25', 'D:hdim7', 'D:1', 'F#:minmaj7', 'D:aug', '<unk>', 'F:maj6', 'ts_15x8', 'C:7(b13)', 'C:dim', 'position_2x25', 'G#:7(#9)', 'D:7(b13)', 'C:min', 'P:24', 'A:dim7', 'F#:7(#11)', 'F#:', 'position_2x75', 'ts_15x8', '

In [14]:
print(output_recon_tokens)

['<h>', 'P:62', 'D:7', 'D#:min', 'position_4x00', 'F#:7', 'G#:9', 'position_1x75', 'A#:minmaj7', 'P:87', 'E:maj9', 'P:77', 'D#:sus2', 'ts_13x8', 'A:dim7', 'F:11', 'P:43', 'F:maj7', 'C:aug', 'P:85', 'G#:7(#9)', 'A:dim7', 'E:min9', 'position_9x83', 'P:85', 'P:40', 'D#:dim', 'D#:min13', 'E:maj9', 'E:dim7', 'P:95', 'G:min7', 'G:11', 'D:min6', 'G:aug', 'P:84', 'C:9', 'A#:13', 'position_9x83', 'C:min9', 'P:36', 'C#:min7', 'G:7', 'C#:min6', 'E:maj6', 'A:13', 'P:107', 'F#:7', 'position_2x66', 'F:aug', 'position_9x83', 'position_8x75', 'A#:sus2', 'D:aug', 'G#:maj9', 'G#:maj9', 'C:dim', 'P:85', 'position_3x33', 'position_7x00', 'P:50', 'D#:min7', '<mask>', 'P:90', 'D:7(b13)', 'G#:maj13', 'D:minmaj7', 'position_1x16', 'C#:hdim7', 'ts_13x8', 'C#:min9', 'E:hdim7', 'F:7(b9)', 'C:maj6', 'A:min', 'position_5x16', 'position_9x83', 'P:21', 'C:min', 'position_3x75', 'position_3x16', 'G#:7(b9)', 'F:min13', 'P:85', 'G:min7', 'F#:sus4', 'F:hdim7', 'P:48', 'P:67', 'G:dim', 'P:37', 'P:66', 'D:min13', 'F:5', '

In [15]:
input_ids = batch['input_ids'].to(device)
print(input_ids)
input_tokens = []
for i in input_ids[0]:
    input_tokens.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )
print(input_tokens)

tensor([[  2,   6, 180,  95,  55, 101,  55, 107,  55, 111,  57, 119,  57,   6,
          95,  58, 101,  57, 107,  53, 111,  55, 119,   4, 123,  53,   6,  95,
          55, 101,  55, 107,  55, 111,  57, 119,  57,   6,  95,  58, 101,  58,
         107,  55, 111,  58, 119,  60]], device='cuda:0')
['<s>', '<bar>', 'ts_4x4', 'position_0x00', 'P:69', 'position_0x75', 'P:69', 'position_1x50', 'P:69', 'position_2x00', 'P:71', 'position_3x00', 'P:71', '<bar>', 'position_0x00', 'P:72', 'position_0x75', 'P:71', 'position_1x50', 'P:67', 'position_2x00', 'P:69', 'position_3x00', '<rest>', 'position_3x50', 'P:67', '<bar>', 'position_0x00', 'P:69', 'position_0x75', 'P:69', 'position_1x50', 'P:69', 'position_2x00', 'P:71', 'position_3x00', 'P:71', '<bar>', 'position_0x00', 'P:72', 'position_0x75', 'P:72', 'position_1x50', 'P:69', 'position_2x00', 'P:72', 'position_3x00', 'P:74']


In [16]:
outputs = bart.generate(
    input_ids=input_ids,
    # attention_mask=batch['attention_mask'][bi],
    bos_token_id=tokenizer.vocab[tokenizer.harmony_tokenizer.start_harmony_token],
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=500,
    do_sample=True,
    temperature=1.0
)

In [17]:
print(outputs)
bart_only_outputs = []
for i in outputs[0]:
    bart_only_outputs.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )
print(bart_only_outputs)

tensor([[  2, 196,   6,  95, 255, 111, 400,   6,  95, 466,   6,  95, 255, 111,
         400,   6,  95, 466,   3]], device='cuda:0')
['<s>', '<h>', '<bar>', 'position_0x00', 'D:maj', 'position_2x00', 'G:maj', '<bar>', 'position_0x00', 'A:min7', '<bar>', 'position_0x00', 'D:maj', 'position_2x00', 'G:maj', '<bar>', 'position_0x00', 'A:min7', '</s>']
